<a href="https://colab.research.google.com/github/yliu9418/LeetCode-Go/blob/master/Twitter.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
import gzip
import shutil
import os
import csv
import linecache
from shutil import copyfile
import ipywidgets as widgets
import numpy as np
import pandas as pd
import wget

CONSUMER_KEY = "LJ5ugdnBADLYRHzuPlI16rzMF"
CONSUMER_SECRET_KEY = "Fj4XtBamfNyP1JE5G5wAPsErJ6y4F3MTlrA5wJQ9J0NsyRgSPr"
ACCESS_TOKEN_KEY = "890342461553422336-hRSVKVUh9wxYfBTCyHJsWANZDX2ATGF"
ACCESS_TOKEN_SECRET_KEY= "OTMlIyuk7pfcQgO64XwmeQPx8Me2A8NKnPiGyAgECuMdi"

# dataset_URL = "https://github.com/thepanacealab/covid19_twitter/blob/master/dailies/2020-11-01/2020-11-01_clean-dataset.tsv.gz?raw=true" #@param {type:"string"}
dataset_URL = "https://github.com/thepanacealab/covid19_twitter/blob/master/dailies/2021-08-01/2021-08-01_clean-dataset.tsv.gz?raw=true"
#Downloads the dataset (compressed in a GZ format)
#!wget dataset_URL -O clean-dataset.tsv.gz
wget.download(dataset_URL, out='clean-dataset.tsv.gz')

#Unzips the dataset and gets the TSV dataset
with gzip.open('clean-dataset.tsv.gz', 'rb') as f_in:
    with open('clean-dataset.tsv', 'wb') as f_out:
        shutil.copyfileobj(f_in, f_out)

#Deletes the compressed GZ file
os.unlink("clean-dataset.tsv.gz")


In [ ]:
#Gets all possible languages from the dataset
df = pd.read_csv('clean-dataset.tsv',sep="\t")
lang_list = df.lang.unique()
lang_list= sorted(np.append(lang_list,'all'))
lang_picker = widgets.Dropdown(options=lang_list, value="en")
lang_picker

#Gets all possible countries from the dataset
#df = pd.read_csv('clean-dataset.tsv',sep="\t")
country_list = df.country_code.unique()
#country_list= sorted(np.append(country_list,'all'))
country_picker = widgets.Dropdown(options=country_list, value="US")
country_picker

Dropdown(index=1, options=(nan, 'US', 'PK', 'CA', 'IN', 'PH', 'TH', 'CO', 'NZ', 'ZA', 'ES', 'ID', 'SV', 'MX', …

In [ ]:
#Creates a new clean dataset with the specified language (if specified)
filtered_language = lang_picker.value
filtered_country = country_picker.value

#If no language specified, it will get all records from the dataset
if filtered_language == "":
  copyfile('clean-dataset.tsv', 'clean-dataset-filtered.tsv')

#If language specified, it will create another tsv file with the filtered records
else:
  filtered_tw = list()
  current_line = 1
  with open("clean-dataset.tsv") as tsvfile:
    tsvreader = csv.reader(tsvfile, delimiter="\t")

    if current_line == 1:
      filtered_tw.append(linecache.getline("clean-dataset.tsv", current_line))

      for line in tsvreader:
        if line[3] == filtered_language and line[4] == filtered_country:
          filtered_tw.append(linecache.getline("clean-dataset.tsv", current_line))
        current_line += 1

  print('\033[1mShowing first 5 tweets from the filtered dataset\033[0m')
  print(filtered_tw[1:(6 if len(filtered_tw) > 6 else len(filtered_tw))])

  with open('clean-dataset-filtered.tsv', 'w') as f_output:
      for item in filtered_tw:
          f_output.write(item)


import json
import tweepy
from tweepy import OAuthHandler


#Creates a JSON Files with the API credentials
with open('api_keys.json', 'w') as outfile:
    json.dump({
    "consumer_key":CONSUMER_KEY,
    "consumer_secret":CONSUMER_SECRET_KEY,
    "access_token":ACCESS_TOKEN_KEY,
    "access_token_secret": ACCESS_TOKEN_SECRET_KEY
     }, outfile)
    

from IPython.display import clear_output

wget.download("https://raw.githubusercontent.com/thepanacealab/SMMT/master/data_acquisition/get_metadata.py", "get_metadata.py")
    
clear_output()

In [ ]:
!python get_metadata.py -i clean-dataset-filtered.tsv -o hydrated_tweets -k api_keys.json -m e

clear_output()

In [ ]:
from IPython.display import clear_output

!wget https://raw.githubusercontent.com/thepanacealab/SMMT/master/data_preprocessing/parse_json_lite.py -O parse_json_lite.py
!wget https://raw.githubusercontent.com/thepanacealab/SMMT/master/data_preprocessing/fields.py -O fields.py

!pip install emot --upgrade
!pip install emoji --upgrade

clear_output()

In [ ]:
import random 

no_samples = "1000" #@param {type:"string"}
list_tweets = None

with open("hydrated_tweets_short.json", "r") as myfile:
    list_tweets = list(myfile)

if int(no_samples) > len(list_tweets):
    no_samples = len(list_tweets)

sample = random.sample(list_tweets, int(no_samples))

file = open("sample_data.json", "w")
for i in sample:
  file.write(i)
file.close() #This close() is important